In [425]:
import vl_convert as vlc
import json
import copy
import pandas as pd

In [426]:
normalize={'date':'date','Date':'date','day':'date','Day':'date','month':'month','Month':'month','year':'year','Year':'year'}
time_format={'date':'D','Date':'D','day':'D','Day':'D','month':'M','Month':'M','year':'Y','Year':'Y'}
def get_opposite_color(hex_color):
    hex_color = hex_color.lstrip('#')
    r = int(hex_color[0:2], 16)
    g = int(hex_color[2:4], 16)
    b = int(hex_color[4:6], 16)
    r = 255 - r
    g = 255 - g
    b = 255 - b
    r_hex = format(r, '02x')
    g_hex = format(g, '02x')
    b_hex = format(b, '02x')
    return f"#{r_hex}{g_hex}{b_hex}"
def remove_lonely_numbers(lst):
    result = []
    lst=sorted(lst)
    for i in range(len(lst)):
        if (i == 0):
            if (lst[i+1] - lst[i] == 1):  
                result.append(lst[i])
        if (i == len(lst) - 1):
            if(lst[i] - lst[i-1] == 1):
                result.append(lst[i])
        if  (i!=0 and i!=len(lst)-1):
            if (lst[i] - lst[i-1] == 1) | (lst[i+1] - lst[i] == 1):
                result.append(lst[i])
    return result

def generate_conditions(numlist,mainField):
    filtered_years = remove_lonely_numbers(numlist)
    print(filtered_years)
    ranges = []
    start = None
    for year in filtered_years:
        if start is None:
            start = year
        elif year + 1 not in filtered_years:
            end = year
            if start != end:
                ranges.append((start, end))
            start = None
    if start is not None:
        end = filtered_years[-1]
        if start != end:
            ranges.append((start, end))
    conditions = [f"({normalize[mainField]}(datum['{mainField}']) >= {start-1} && {normalize[mainField]}(datum['{mainField}']) <= {end-1})" for start, end in ranges ]
    return conditions

def LineMark(vega,mainField, subField, mainType, subType, value):
    newcolor = get_opposite_color(vega["encoding"]["color"]["value"])
    if mainType == 'temporal':
        vega["data"]["format"]= {"type": "csv", "parse": {f'{mainField}': f"date:'%{time_format[mainField]}'"}}
        vega["layer"].append( {
        "mark": {"type": "point", "filled": True},
        "transform": [{"filter": f"{normalize[mainField]}(datum['{mainField}']) == {value}"}],
        "encoding": {
            "color": {"value": newcolor},
            "x": {"type": mainType, "field": mainField},
            "y": {"type": subType, "field": subField}
        }})
    else:
        vega["layer"].append( {
            "mark": {"type": "point", "filled": True},
            "transform": [{"filter": f"datum['{mainField}'] == {value}"}], # type: ignore
            "encoding": {
                "color": {"value": newcolor},
                "x": {"type": mainType, "field": mainField},
                "y": {"type": subType, "field": subField}
            }})
    return vega

def LineText(vega,mainField, subField, mainType, subType, value, text,degree=0):
    newcolor = get_opposite_color(vega["encoding"]["color"]["value"])
    size=14
    if degree!=0:
        size=32
    if mainType == 'temporal':
        vega["data"]["format"]= {"type": "csv", "parse": {f'{mainField}': f"date:'%{time_format[mainField]}'"}}
        vega["layer"].append( {
        "mark": {"type": "text", "filled": True, "angle": degree,"fontSize": size,'dy':-10},
        "transform": [{"filter": f"{normalize[mainField]}(datum['{mainField}']) == {value}"}],
        "encoding": {
            "text" :{"value": text},
            "color": {"value": newcolor},
            "x": {"type": mainType, "field": mainField},
            "y": {"type": subType, "field": subField}
        }})
    else:
        vega["layer"].append({
            "mark": {"type": "text", "filled": True, "angle": degree,'dy':-10},
            "transform": [{"filter": f"datum['{mainField}'] == {value}"}], # type: ignore
            "encoding": {
                "text" :{"value": text},
                "color": {"value": newcolor},
                "x": {"type": mainType, "field": mainField},
                "y": {"type": subType, "field": subField}
            }})
    return vega

def LineLine(vega,mainField,subField, mainType, subType, value,value1=0,subValue=0,subValue1=0):
    newcolor = get_opposite_color(vega["encoding"]["color"]["value"])
    if subValue==0:
        if vega["encoding"]["x"]["field"] == subField:
            vega["layer"].append({ "data": { "values": [{}] }, "mark": { "type": "rule", "color": newcolor }, "encoding": { "x": { "datum": value } } })
            if value1!=0:
                vega["layer"].append({ "data": { "values": [{}] }, "mark": { "type": "rule", "color": newcolor }, "encoding": { "x": { "datum": value1 } } })
        else:
            vega["layer"].append({ "data": { "values": [{}] }, "mark": { "type": "rule", "color": newcolor }, "encoding": { "y": { "datum": value } } })
            if value1!=0:
                vega["layer"].append({ "data": { "values": [{}] }, "mark": { "type": "rule", "color": newcolor }, "encoding": { "y": { "datum": value1 } } })
    else:
        vega["layer"].append({
      "data": {
        "values": [
          {"PlotX1": value, "PlotY1": value1, "PlotX2": subValue, "PlotY2": subValue1}
        ]
      },
      "mark": {
        "type": "rule",
        "stroke": newcolor,
        "strokeWidth": 2
      },
      "encoding": {
        "x": {"field": "PlotX1", "type": mainType},
        "y": {"field": "PlotY1", "type": subType},
        "x2": {"field": "PlotX2", "type": mainType},
        "y2": {"field": "PlotY2", "type": subType}
      }
    })
    return vega
def LineColor(vega,mainField, subField, mainType, subType, value,condition_type,value1=0):
    newcolor = get_opposite_color(vega["encoding"]["color"]["value"])
    df = pd.read_csv(vega["data"]["url"])
    if condition_type =='left_bound' or condition_type =='right_bound' or condition_type=='left_and_right':
        new_layer = copy.deepcopy(vega["layer"][0])
        if condition_type =='left_bound':
            new_layer["transform"] = [{"filter": f"{normalize[mainField]}(datum['{mainField}']) >= {value-1}"}]
        elif condition_type =='right_bound':
            new_layer["transform"] = [{"filter": f"{normalize[mainField]}(datum['{mainField}']) <= {value-1}"}]
        elif condition_type == 'left_and_right':
            new_layer["transform"] = [{"filter": f"{normalize[mainField]}(datum['{mainField}']) >= {value-1} && {normalize[mainField]}(datum['{mainField}']) <= {value1-1}"}]
        new_layer["encoding"]["color"]["value"] = newcolor
        vega["layer"].append(new_layer)
    else:
        if condition_type=='above':
            indices = df[df.columns.tolist()[0]][df[df[df.columns.tolist()[1]] > value].index].tolist()
        if condition_type=='below':
            indices = df[df.columns.tolist()[0]][df[df[df.columns.tolist()[1]] < value].index].tolist()
        if condition_type=='above_below':
            indices = df[df.columns.tolist()[0]][df[(df[df.columns.tolist()[1]] > value) & (df[df.columns.tolist()[1]] < value1)].index].tolist()	
        conditions=generate_conditions(indices,mainField)
        for condition in conditions:
            new_layer = copy.deepcopy(vega["layer"][0])
            new_layer["transform"] = [{"filter": condition}]
           	
            vega["layer"].append(new_layer)
    return vega
def LineOpacity(vega,mainField, subField, mainType, subType, value,condition_type,value1=0):
    df = pd.read_csv(vega["data"]["url"])
    vega["layer"][0]["encoding"]["opacity"]={"value": 0.4}
    if condition_type =='left_bound' or condition_type =='right_bound' or condition_type=='left_and_right':
            new_layer = copy.deepcopy(vega["layer"][0])
            if condition_type =='left_bound':
                new_layer["transform"] = [{"filter": f"{normalize[mainField]}(datum['{mainField}']) >= {value-1}"}]
            elif condition_type =='right_bound':
                new_layer["transform"] = [{"filter": f"{normalize[mainField]}(datum['{mainField}']) <= {value-1}"}]
            elif condition_type == 'left_and_right':
                new_layer["transform"] = [{"filter": f"{normalize[mainField]}(datum['{mainField}']) >= {value-1} && {normalize[mainField]}(datum['{mainField}']) <= {value1-1}"}]
            new_layer["encoding"]["opacity"]={"value": 1}
            vega["layer"].append(new_layer)
    else:
        if condition_type=='above':
            indices = df[df.columns.tolist()[0]][df[df[df.columns.tolist()[1]] > value].index].tolist()
        if condition_type=='below':
            indices = df[df.columns.tolist()[0]][df[df[df.columns.tolist()[1]] < value].index].tolist()
        if condition_type=='above_below':
            indices = df[df.columns.tolist()[0]][df[(df[df.columns.tolist()[1]] > value) & (df[df.columns.tolist()[1]] < value1)].index].tolist()	
        conditions=generate_conditions(indices,mainField)
        for condition in conditions:
            new_layer = copy.deepcopy(vega["layer"][0])
            new_layer["transform"] = [{"filter": condition}]
            new_layer["encoding"]["opacity"]={"value": 1}
            vega["layer"].append(new_layer)
    return vega
def LineArea(vega,mainField, subField, mainType, subType, value,value1=0,subValue=0,subValue1=0,shade=True):
    newcolor = get_opposite_color(vega["encoding"]["color"]["value"])
    # df = pd.read_csv(vega["data"]["url"])
    if shade==True:
        stroke=0
        opacity=0.2
    else:
        stroke=2
        opacity=0
    vega["layer"].append({
      "data": {
        "values": [
          {"PlotX1": value, "PlotY1": value1, "PlotX2": subValue, "PlotY2": subValue1}
        ]
      },
        "mark": {"type": "rect", "color": newcolor,"stroke":newcolor,"fillOpacity":opacity,"strokeWidth":stroke},
      "encoding": {
        "x": {"field": "PlotX1", "type": mainType},
        "y": {"field": "PlotY1", "type": subType},
        "x2": {"field": "PlotX2", "type": mainType},
        "y2": {"field": "PlotY2", "type": subType}
    }})
    return vega

In [427]:
vl_spec=r"""{
  "config": {
    "area": {"fill": "#30a2da"},
    "axis": {
      "domainColor": "#cbcbcb",
      "grid": true,
      "gridColor": "#cbcbcb",
      "gridWidth": 1,
      "labelColor": "#999",
      "labelFontSize": 10,
      "titleColor": "#333",
      "tickColor": "#cbcbcb",
      "tickSize": 10,
      "titleFontSize": 14,
      "titlePadding": 10,
      "labelPadding": 4
    },
    "axisBand": {"grid": false},
    "background": "#f0f0f0",
    "view": {"fill": "#f0f0f0"},
    "legend": {
      "labelColor": "#333",
      "labelFontSize": 11,
      "padding": 1,
      "symbolSize": 30,
      "symbolType": "square",
      "titleColor": "#333",
      "titleFontSize": 14,
      "titlePadding": 10
    },
    "line": {"stroke": "#30a2da", "strokeWidth": 2},
    "rect": {"fill": "#30a2da"},
    "bar": {"binSpacing": 2, "fill": "#30a2da", "stroke": null},
    "range": {
      "category": [
        "#30a2da",
        "#fc4f30",
        "#e5ae38",
        "#6d904f",
        "#8b8b8b",
        "#b96db8",
        "#ff9e27",
        "#56cc60",
        "#52d2ca",
        "#52689e",
        "#545454",
        "#9fe4f8"
      ],
      "diverging": [
        "#cc0020",
        "#e77866",
        "#f6e7e1",
        "#d6e8ed",
        "#91bfd9",
        "#1d78b5"
      ],
      "heatmap": ["#d6e8ed", "#cee0e5", "#91bfd9", "#549cc6", "#1d78b5"]
    },
    "point": {"filled": true, "shape": "circle"},
    "title": {
      "anchor": "start",
      "fontSize": 24,
      "fontWeight": 600,
      "offset": 20
    }
  },
  "data": {
    "url": "https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10176.csv"
  },
  "mark": "line",
  "encoding": {
    "color": {"value": "#ff9e27"},
    "x": {"type": "temporal", "axis": {}, "bin": false, "field": "Year"},
    "y": {
      "type": "quantitative",
      "axis": {
        "labelAngle": 30,
        "title": "Operating revenue in million U.S. dollars"
      },
      "field": "Operating revenue in million U\\.S\\. dollars"
    }
  },
  "title": [
    "Waste Management Inc. 's operating revenue",
    "from FY 2007 to FY 2019 (in million U.S.",
    "dollars)"
  ],
  "$schema": "https://vega.github.io/schema/vega-lite/v4.8.1.json"
}"""

In [428]:
png_data = vlc.vegalite_to_png(vl_spec=vl_spec, scale=2)
with open("Sample-Line-RetrieveValue.png", "wb") as f:
    f.write(png_data)

In [429]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=LineMark(vl_json,"Year", "Operating revenue in million U\\.S\\. dollars", "temporal", "quantitative", 2007)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("RetrieveValue-Line-Mark.png", "wb") as f:
    f.write(png_data)

{'config': {'area': {'fill': '#30a2da'}, 'axis': {'domainColor': '#cbcbcb', 'grid': True, 'gridColor': '#cbcbcb', 'gridWidth': 1, 'labelColor': '#999', 'labelFontSize': 10, 'titleColor': '#333', 'tickColor': '#cbcbcb', 'tickSize': 10, 'titleFontSize': 14, 'titlePadding': 10, 'labelPadding': 4}, 'axisBand': {'grid': False}, 'background': '#f0f0f0', 'view': {'fill': '#f0f0f0'}, 'legend': {'labelColor': '#333', 'labelFontSize': 11, 'padding': 1, 'symbolSize': 30, 'symbolType': 'square', 'titleColor': '#333', 'titleFontSize': 14, 'titlePadding': 10}, 'line': {'stroke': '#30a2da', 'strokeWidth': 2}, 'rect': {'fill': '#30a2da'}, 'bar': {'binSpacing': 2, 'fill': '#30a2da', 'stroke': None}, 'range': {'category': ['#30a2da', '#fc4f30', '#e5ae38', '#6d904f', '#8b8b8b', '#b96db8', '#ff9e27', '#56cc60', '#52d2ca', '#52689e', '#545454', '#9fe4f8'], 'diverging': ['#cc0020', '#e77866', '#f6e7e1', '#d6e8ed', '#91bfd9', '#1d78b5'], 'heatmap': ['#d6e8ed', '#cee0e5', '#91bfd9', '#549cc6', '#1d78b5']}, 'p

In [430]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=LineText(vl_json,"Year", "Operating revenue in million U\\.S\\. dollars", "temporal", "quantitative", 2007,"<15million")
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("RetrieveValue-Line-Text.png", "wb") as f:
    f.write(png_data)

{'config': {'area': {'fill': '#30a2da'}, 'axis': {'domainColor': '#cbcbcb', 'grid': True, 'gridColor': '#cbcbcb', 'gridWidth': 1, 'labelColor': '#999', 'labelFontSize': 10, 'titleColor': '#333', 'tickColor': '#cbcbcb', 'tickSize': 10, 'titleFontSize': 14, 'titlePadding': 10, 'labelPadding': 4}, 'axisBand': {'grid': False}, 'background': '#f0f0f0', 'view': {'fill': '#f0f0f0'}, 'legend': {'labelColor': '#333', 'labelFontSize': 11, 'padding': 1, 'symbolSize': 30, 'symbolType': 'square', 'titleColor': '#333', 'titleFontSize': 14, 'titlePadding': 10}, 'line': {'stroke': '#30a2da', 'strokeWidth': 2}, 'rect': {'fill': '#30a2da'}, 'bar': {'binSpacing': 2, 'fill': '#30a2da', 'stroke': None}, 'range': {'category': ['#30a2da', '#fc4f30', '#e5ae38', '#6d904f', '#8b8b8b', '#b96db8', '#ff9e27', '#56cc60', '#52d2ca', '#52689e', '#545454', '#9fe4f8'], 'diverging': ['#cc0020', '#e77866', '#f6e7e1', '#d6e8ed', '#91bfd9', '#1d78b5'], 'heatmap': ['#d6e8ed', '#cee0e5', '#91bfd9', '#549cc6', '#1d78b5']}, 'p

In [431]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=LineLine(vl_json,"Year", "Operating revenue in million U\\.S\\. dollars", "temporal", "quantitative", 15000)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("RetrieveValue-Line-Line.png", "wb") as f:
    f.write(png_data)

{'config': {'area': {'fill': '#30a2da'}, 'axis': {'domainColor': '#cbcbcb', 'grid': True, 'gridColor': '#cbcbcb', 'gridWidth': 1, 'labelColor': '#999', 'labelFontSize': 10, 'titleColor': '#333', 'tickColor': '#cbcbcb', 'tickSize': 10, 'titleFontSize': 14, 'titlePadding': 10, 'labelPadding': 4}, 'axisBand': {'grid': False}, 'background': '#f0f0f0', 'view': {'fill': '#f0f0f0'}, 'legend': {'labelColor': '#333', 'labelFontSize': 11, 'padding': 1, 'symbolSize': 30, 'symbolType': 'square', 'titleColor': '#333', 'titleFontSize': 14, 'titlePadding': 10}, 'line': {'stroke': '#30a2da', 'strokeWidth': 2}, 'rect': {'fill': '#30a2da'}, 'bar': {'binSpacing': 2, 'fill': '#30a2da', 'stroke': None}, 'range': {'category': ['#30a2da', '#fc4f30', '#e5ae38', '#6d904f', '#8b8b8b', '#b96db8', '#ff9e27', '#56cc60', '#52d2ca', '#52689e', '#545454', '#9fe4f8'], 'diverging': ['#cc0020', '#e77866', '#f6e7e1', '#d6e8ed', '#91bfd9', '#1d78b5'], 'heatmap': ['#d6e8ed', '#cee0e5', '#91bfd9', '#549cc6', '#1d78b5']}, 'p

In [432]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=LineColor(vl_json,"Year", "Operating revenue in million U\\.S\\. dollars", "temporal", "quantitative", 2007,"left_and_right",2008)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("RetrieveValue-Line-Color.png", "wb") as f:
    f.write(png_data)

{'config': {'area': {'fill': '#30a2da'}, 'axis': {'domainColor': '#cbcbcb', 'grid': True, 'gridColor': '#cbcbcb', 'gridWidth': 1, 'labelColor': '#999', 'labelFontSize': 10, 'titleColor': '#333', 'tickColor': '#cbcbcb', 'tickSize': 10, 'titleFontSize': 14, 'titlePadding': 10, 'labelPadding': 4}, 'axisBand': {'grid': False}, 'background': '#f0f0f0', 'view': {'fill': '#f0f0f0'}, 'legend': {'labelColor': '#333', 'labelFontSize': 11, 'padding': 1, 'symbolSize': 30, 'symbolType': 'square', 'titleColor': '#333', 'titleFontSize': 14, 'titlePadding': 10}, 'line': {'stroke': '#30a2da', 'strokeWidth': 2}, 'rect': {'fill': '#30a2da'}, 'bar': {'binSpacing': 2, 'fill': '#30a2da', 'stroke': None}, 'range': {'category': ['#30a2da', '#fc4f30', '#e5ae38', '#6d904f', '#8b8b8b', '#b96db8', '#ff9e27', '#56cc60', '#52d2ca', '#52689e', '#545454', '#9fe4f8'], 'diverging': ['#cc0020', '#e77866', '#f6e7e1', '#d6e8ed', '#91bfd9', '#1d78b5'], 'heatmap': ['#d6e8ed', '#cee0e5', '#91bfd9', '#549cc6', '#1d78b5']}, 'p

In [433]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=LineOpacity(vl_json,"Year", "Operating revenue in million U\\.S\\. dollars", "temporal", "quantitative", 2007,"left_and_right",2008)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("RetrieveValue-Line-Opacity.png", "wb") as f:
    f.write(png_data)

{'config': {'area': {'fill': '#30a2da'}, 'axis': {'domainColor': '#cbcbcb', 'grid': True, 'gridColor': '#cbcbcb', 'gridWidth': 1, 'labelColor': '#999', 'labelFontSize': 10, 'titleColor': '#333', 'tickColor': '#cbcbcb', 'tickSize': 10, 'titleFontSize': 14, 'titlePadding': 10, 'labelPadding': 4}, 'axisBand': {'grid': False}, 'background': '#f0f0f0', 'view': {'fill': '#f0f0f0'}, 'legend': {'labelColor': '#333', 'labelFontSize': 11, 'padding': 1, 'symbolSize': 30, 'symbolType': 'square', 'titleColor': '#333', 'titleFontSize': 14, 'titlePadding': 10}, 'line': {'stroke': '#30a2da', 'strokeWidth': 2}, 'rect': {'fill': '#30a2da'}, 'bar': {'binSpacing': 2, 'fill': '#30a2da', 'stroke': None}, 'range': {'category': ['#30a2da', '#fc4f30', '#e5ae38', '#6d904f', '#8b8b8b', '#b96db8', '#ff9e27', '#56cc60', '#52d2ca', '#52689e', '#545454', '#9fe4f8'], 'diverging': ['#cc0020', '#e77866', '#f6e7e1', '#d6e8ed', '#91bfd9', '#1d78b5'], 'heatmap': ['#d6e8ed', '#cee0e5', '#91bfd9', '#549cc6', '#1d78b5']}, 'p

In [434]:
vl_spec=r'''
{
  "config": {
    "background": "#fff",
    "area": {"fill": "#4572a7"},
    "line": {"stroke": "#4572a7", "strokeWidth": 2},
    "rect": {"fill": "#4572a7"},
    "bar": {"fill": "#4572a7"},
    "point": {"color": "#4572a7", "strokeWidth": 1.5, "size": 50},
    "axis": {
      "bandPosition": 0.5,
      "gridColor": "#000000",
      "gridOpacity": 1,
      "gridWidth": 0.5,
      "labelPadding": 10,
      "tickSize": 5,
      "tickWidth": 0.5
    },
    "axisBand": {},
    "legend": {
      "labelBaseline": "middle",
      "labelFontSize": 11,
      "symbolSize": 50,
      "symbolType": "square"
    },
    "range": {
      "category": [
        "#4572a7",
        "#aa4643",
        "#8aa453",
        "#71598e",
        "#4598ae",
        "#d98445",
        "#94aace",
        "#d09393",
        "#b9cc98",
        "#a99cbc"
      ]
    }
  },
  "data": {
    "url": "https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10061.csv"
  },
  "mark": "line",
  "encoding": {
    "color": {"value": "#d98445"},
    "x": {"type": "temporal", "axis": {}, "field": "Year"},
    "y": {
      "type": "quantitative",
      "axis": {"labelAngle": 30, "title": "Number of hospitals"},
      "field": "Number of hospitals"
    }
  },
  "title": [
    "Number of hospitals in the United States",
    "from 1999 to 2017 (in thousands)"
  ],
  "$schema": "https://vega.github.io/schema/vega-lite/v4.8.1.json"
}
'''

In [435]:
png_data = vlc.vegalite_to_png(vl_spec=vl_spec, scale=2)
with open("Sample-Line-Filter.png", "wb") as f:
    f.write(png_data)

In [436]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=LineColor(vl_json,"Year", "Number of hospitals", "temporal", "quantitative", 3500,'below')
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Filter-Line-Color.png", "wb") as f:
    f.write(png_data)

[2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10061.csv'}, 'title': ['Number of hospitals in the United States', 'from 1999 to 2017 (in thousands)'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer'

In [437]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=LineOpacity(vl_json,"Year", "Number of hospitals", "temporal", "quantitative", 3500,'below')
# print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Filter-Line-Opacity.png", "wb") as f:
    f.write(png_data)

[2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]


In [438]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=LineLine(vl_json,"Year", "Number of hospitals", "temporal", "quantitative", 3500)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Filter-Line-Line.png", "wb") as f:
    f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10061.csv'}, 'title': ['Number of hospitals in the United States', 'from 1999 to 2017 (in thousands)'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'line', 'encoding': {'color': {'value': '#d98445'}, 'x': {'type': 'tempor

In [439]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=LineArea(vl_json,"Year", "Number of hospitals", "temporal", "quantitative", "2000",0,"2017",3500)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Filter-Line-Shade.png", "wb") as f:
    f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10061.csv'}, 'title': ['Number of hospitals in the United States', 'from 1999 to 2017 (in thousands)'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'line', 'encoding': {'color': {'value': '#d98445'}, 'x': {'type': 'tempor

In [440]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=LineArea(vl_json,"Year", "Operating revenue in million U\\.S\\. dollars", "temporal", "quantitative", "2000",0,"2017",3500,shade=False)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Filter-Line-Bound.png", "wb") as f:
    f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10061.csv'}, 'title': ['Number of hospitals in the United States', 'from 1999 to 2017 (in thousands)'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'line', 'encoding': {'color': {'value': '#d98445'}, 'x': {'type': 'tempor

In [441]:
vl_spec=r'''
{
  "config": {
    "background": "#fff",
    "area": {"fill": "#4572a7"},
    "line": {"stroke": "#4572a7", "strokeWidth": 2},
    "rect": {"fill": "#4572a7"},
    "bar": {"fill": "#4572a7"},
    "point": {"color": "#4572a7", "strokeWidth": 1.5, "size": 50},
    "axis": {
      "bandPosition": 0.5,
      "gridColor": "#000000",
      "gridOpacity": 1,
      "gridWidth": 0.5,
      "labelPadding": 10,
      "tickSize": 5,
      "tickWidth": 0.5
    },
    "axisBand": {},
    "legend": {
      "labelBaseline": "middle",
      "labelFontSize": 11,
      "symbolSize": 50,
      "symbolType": "square"
    },
    "range": {
      "category": [
        "#4572a7",
        "#aa4643",
        "#8aa453",
        "#71598e",
        "#4598ae",
        "#d98445",
        "#94aace",
        "#d09393",
        "#b9cc98",
        "#a99cbc"
      ]
    }
  },
  "data": {
    "url": "https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10520.csv"
  },
  "mark": "line",
  "encoding": {
    "color": {"value": "#d98445"},
    "x": {"type": "temporal", "axis": {}, "field": "Year"},
    "y": {
      "type": "quantitative",
      "axis": {"labelAngle": -45, "title": "Unit sales in millions"},
      "field": "Unit sales in millions"
    }
  },
  "title": ["Unit sales of the world"]
}
'''

In [442]:
png_data = vlc.vegalite_to_png(vl_spec=vl_spec, scale=2)
with open("Sample-Line-Range.png", "wb") as f:
    f.write(png_data)

In [443]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=LineColor(vl_json,"Year", "Unit sales in millions", "temporal", "quantitative", 0.5,'above_below',1.4)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Range-Line-Color.png", "wb") as f:
    f.write(png_data)

[2015, 2016, 2017, 2018]
{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10520.csv'}, 'title': ['Unit sales of the world'], 'layer': [{'mark': 'line', 'encoding': {'color': {'value': '#d98445'}, 'x': {'type': 'temporal', 'axis': {}, 'field': 'Year'}, 'y': {'type': 'quantitative', 'axis': {'labelAngle': -45, '

In [444]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=LineOpacity(vl_json,"Year", "Unit sales in millions", "temporal", "quantitative", 0.5,'above_below',1.4)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Range-Line-Opacity.png", "wb") as f:
    f.write(png_data)

[2015, 2016, 2017, 2018]
{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10520.csv'}, 'title': ['Unit sales of the world'], 'layer': [{'mark': 'line', 'encoding': {'color': {'value': '#d98445'}, 'x': {'type': 'temporal', 'axis': {}, 'field': 'Year'}, 'y': {'type': 'quantitative', 'axis': {'labelAngle': -45, '

In [445]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=LineLine(vl_json,"Year", "Unit sales in millions", "temporal", "quantitative", 0.5,1.4)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Range-Line-Line.png", "wb") as f:
    f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10520.csv'}, 'title': ['Unit sales of the world'], 'layer': [{'mark': 'line', 'encoding': {'color': {'value': '#d98445'}, 'x': {'type': 'temporal', 'axis': {}, 'field': 'Year'}, 'y': {'type': 'quantitative', 'axis': {'labelAngle': -45, 'title': 'Unit sales in mi

In [446]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=LineArea(vl_json,"Year", "Unit sales in millions", "temporal", "quantitative", "2008",0.5,"2018",1.4)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Range-Line-Shade.png", "wb") as f:
    f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10520.csv'}, 'title': ['Unit sales of the world'], 'layer': [{'mark': 'line', 'encoding': {'color': {'value': '#d98445'}, 'x': {'type': 'temporal', 'axis': {}, 'field': 'Year'}, 'y': {'type': 'quantitative', 'axis': {'labelAngle': -45, 'title': 'Unit sales in mi

In [447]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=LineArea(vl_json,"Year", "Unit sales in millions", "temporal", "quantitative", "2008",0.5,"2018",1.4,shade=False)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Range-Line-Bound.png", "wb") as f:
    f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10520.csv'}, 'title': ['Unit sales of the world'], 'layer': [{'mark': 'line', 'encoding': {'color': {'value': '#d98445'}, 'x': {'type': 'temporal', 'axis': {}, 'field': 'Year'}, 'y': {'type': 'quantitative', 'axis': {'labelAngle': -45, 'title': 'Unit sales in mi

In [448]:
vl_spec=r''' 
{
  "config": {
    "background": "#fff",
    "area": {"fill": "#4572a7"},
    "line": {"stroke": "#4572a7", "strokeWidth": 2},
    "rect": {"fill": "#4572a7"},
    "bar": {"fill": "#4572a7"},
    "point": {"color": "#4572a7", "strokeWidth": 1.5, "size": 50},
    "axis": {
      "bandPosition": 0.5,
      "gridColor": "#000000",
      "gridOpacity": 1,
      "gridWidth": 0.5,
      "labelPadding": 10,
      "tickSize": 5,
      "tickWidth": 0.5
    },
    "axisBand": {},
    "legend": {
      "labelBaseline": "middle",
      "labelFontSize": 11,
      "symbolSize": 50,
      "symbolType": "square"
    },
    "range": {
      "category": [
        "#4572a7",
        "#aa4643",
        "#8aa453",
        "#71598e",
        "#4598ae",
        "#d98445",
        "#94aace",
        "#d09393",
        "#b9cc98",
        "#a99cbc"
      ]
    }
  },
  "data": {
    "url": "https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10736.csv"
  },
  "mark": "line",
  "encoding": {
    "color": {"value": "#d98445"},
    "x": {"type": "temporal", "axis": {}, "field": "Year"},
    "y": {
      "type": "quantitative",
      "axis": {"labelAngle": 30, "title": "Per capita consumption in pounds"},
      "field": "Per capita consumption in pounds"
    }
  },
  "title": [
    "Per capita consumption of alcoholic",
    "beverages in the United Kingdom from 1999 to",
    "2019"
  ],
  "$schema": "https://vega.github.io/schema/vega-lite/v4.8.1.json"
}
'''

In [449]:
png_data = vlc.vegalite_to_png(vl_spec=vl_spec, scale=2)
with open("Sample-Line-Trend.png", "wb") as f:
    f.write(png_data)

In [450]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=LineColor(vl_json,"Year", "Per capita consumption in pounds", "temporal", "quantitative", 2010,'left_and_right',2020)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Trend-Line-Color.png", "wb") as f:
    f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10736.csv'}, 'title': ['Per capita consumption of alcoholic', 'beverages in the United Kingdom from 1999 to', '2019'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'line', 'encoding': {'color': {'value': '#d98445'}, 'x': {

In [451]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=LineOpacity(vl_json,"Year", "Per capita consumption in pounds", "temporal", "quantitative", 2010,'left_and_right',2020)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Trend-Line-Opacity.png", "wb") as f:
    f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10736.csv'}, 'title': ['Per capita consumption of alcoholic', 'beverages in the United Kingdom from 1999 to', '2019'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'line', 'encoding': {'color': {'value': '#d98445'}, 'x': {

In [452]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=LineText(vl_json,"Year", "Per capita consumption in pounds", "temporal", "quantitative", 2015,"→",30)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Trend-Line-Mark.png", "wb") as f:
    f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10736.csv', 'format': {'type': 'csv', 'parse': {'Year': "date:'%Y'"}}}, 'title': ['Per capita consumption of alcoholic', 'beverages in the United Kingdom from 1999 to', '2019'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark':

In [453]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=LineText(vl_json,"Year", "Per capita consumption in pounds", "temporal", "quantitative", 2015,"decrease")
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Trend-Line-Text.png", "wb") as f:
    f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10736.csv', 'format': {'type': 'csv', 'parse': {'Year': "date:'%Y'"}}}, 'title': ['Per capita consumption of alcoholic', 'beverages in the United Kingdom from 1999 to', '2019'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark':

In [454]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=LineLine(vl_json,"Year", "Per capita consumption in pounds", "temporal", "quantitative", "2010",9,"2019",6.5)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Trend-Line-Line.png", "wb") as f:
    f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10736.csv'}, 'title': ['Per capita consumption of alcoholic', 'beverages in the United Kingdom from 1999 to', '2019'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'line', 'encoding': {'color': {'value': '#d98445'}, 'x': {

In [455]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=LineArea(vl_json,"Year", "Per capita consumption in pounds", "temporal", "quantitative", "2010",6,"2019",10)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Trend-Line-Shade.png", "wb") as f:
    f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10736.csv'}, 'title': ['Per capita consumption of alcoholic', 'beverages in the United Kingdom from 1999 to', '2019'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'line', 'encoding': {'color': {'value': '#d98445'}, 'x': {

In [456]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=LineArea(vl_json,"Year", "Per capita consumption in pounds", "temporal", "quantitative", "2010",6,"2019",10,shade=False)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Trend-Line-Bound.png", "wb") as f:
    f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/10736.csv'}, 'title': ['Per capita consumption of alcoholic', 'beverages in the United Kingdom from 1999 to', '2019'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'line', 'encoding': {'color': {'value': '#d98445'}, 'x': {

In [457]:
vl_spec=r'''
{
  "config": {
    "background": "#fff",
    "area": {"fill": "#4572a7"},
    "line": {"stroke": "#4572a7", "strokeWidth": 2},
    "rect": {"fill": "#4572a7"},
    "bar": {"fill": "#4572a7"},
    "point": {"color": "#4572a7", "strokeWidth": 1.5, "size": 50},
    "axis": {
      "bandPosition": 0.5,
      "gridColor": "#000000",
      "gridOpacity": 1,
      "gridWidth": 0.5,
      "labelPadding": 10,
      "tickSize": 5,
      "tickWidth": 0.5
    },
    "axisBand": {},
    "legend": {
      "labelBaseline": "middle",
      "labelFontSize": 11,
      "symbolSize": 50,
      "symbolType": "square"
    },
    "range": {
      "category": [
        "#4572a7",
        "#aa4643",
        "#8aa453",
        "#71598e",
        "#4598ae",
        "#d98445",
        "#94aace",
        "#d09393",
        "#b9cc98",
        "#a99cbc"
      ]
    }
  },
  "data": {
    "url": "https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/12287.csv"
  },
  "mark": "line",
  "encoding": {
    "color": {"value": "#d98445"},
    "x": {"type": "temporal", "axis": {}, "field": "Year"},
    "y": {
      "type": "quantitative",
      "axis": {
        "labelAngle": -45,
        "title": "Operating profit in million Swiss francs"
      },
      "field": "Operating profit in million Swiss francs"
    }
  },
  "title": [
    "Operating profit of Roche Group from 2007 to 2019",
    "(in million Swiss francs)"
  ],
  "$schema": "https://vega.github.io/schema/vega-lite/v4.8.1.json"
}
'''

In [458]:
png_data = vlc.vegalite_to_png(vl_spec=vl_spec, scale=2)
with open("Sample-Line-Compare.png", "wb") as f:
    f.write(png_data)

In [459]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=LineMark(vl_json,"Year", "Operating profit in million Swiss francs", "temporal", "quantitative", 2008)
vl_json=LineMark(vl_json,"Year", "Operating profit in million Swiss francs", "temporal", "quantitative", 2018)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Compare-Line-Mark.png", "wb") as f:
    f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/12287.csv', 'format': {'type': 'csv', 'parse': {'Year': "date:'%Y'"}}}, 'title': ['Operating profit of Roche Group from 2007 to 2019', '(in million Swiss francs)'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'line', 'enc

In [460]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=LineText(vl_json,"Year", "Operating profit in million Swiss francs", "temporal", "quantitative", 2008,"lower")
vl_json=LineText(vl_json,"Year", "Operating profit in million Swiss francs", "temporal", "quantitative", 2018,"higher")
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Compare-Line-Text.png", "wb") as f:
    f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/12287.csv', 'format': {'type': 'csv', 'parse': {'Year': "date:'%Y'"}}}, 'title': ['Operating profit of Roche Group from 2007 to 2019', '(in million Swiss francs)'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'line', 'enc

In [461]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=LineColor(vl_json,"Year", "Operating profit in million Swiss francs", "temporal", "quantitative", 2008,'left_and_right',2009)
vl_json=LineColor(vl_json,"Year", "Operating profit in million Swiss francs", "temporal", "quantitative", 2018,'left_and_right',2019)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Compare-Line-Color.png", "wb") as f:
    f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/12287.csv'}, 'title': ['Operating profit of Roche Group from 2007 to 2019', '(in million Swiss francs)'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'line', 'encoding': {'color': {'value': '#d98445'}, 'x': {'type': 'temp

In [462]:
vl_json=json.loads(vl_spec)
vl_json["layer"] = [{ 'mark': vl_json['mark'], 'encoding': vl_json['encoding'] }]
vl_json=LineOpacity(vl_json,"Year", "Operating profit in million Swiss francs", "temporal", "quantitative", 2008,'left_and_right',2009)
vl_json=LineOpacity(vl_json,"Year", "Operating profit in million Swiss francs", "temporal", "quantitative", 2018,'left_and_right',2019)
del vl_json['mark'], vl_json['encoding']
print(vl_json)
png_data = vlc.vegalite_to_png(vl_spec=vl_json, scale=2)
with open("Compare-Line-Opacity.png", "wb") as f:
    f.write(png_data)

{'config': {'background': '#fff', 'area': {'fill': '#4572a7'}, 'line': {'stroke': '#4572a7', 'strokeWidth': 2}, 'rect': {'fill': '#4572a7'}, 'bar': {'fill': '#4572a7'}, 'point': {'color': '#4572a7', 'strokeWidth': 1.5, 'size': 50}, 'axis': {'bandPosition': 0.5, 'gridColor': '#000000', 'gridOpacity': 1, 'gridWidth': 0.5, 'labelPadding': 10, 'tickSize': 5, 'tickWidth': 0.5}, 'axisBand': {}, 'legend': {'labelBaseline': 'middle', 'labelFontSize': 11, 'symbolSize': 50, 'symbolType': 'square'}, 'range': {'category': ['#4572a7', '#aa4643', '#8aa453', '#71598e', '#4598ae', '#d98445', '#94aace', '#d09393', '#b9cc98', '#a99cbc']}}, 'data': {'url': 'https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/12287.csv'}, 'title': ['Operating profit of Roche Group from 2007 to 2019', '(in million Swiss francs)'], '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json', 'layer': [{'mark': 'line', 'encoding': {'color': {'value': '#d98445'}, 'x': {'type': 'temp